In [19]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import model_from_json
import os, sklearn, pandas, numpy as np, random
from sklearn import svm
import skimage, skimage.io, skimage.filters
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
from sklearn.utils import shuffle
import imp
from sklearn.preprocessing import LabelBinarizer
# from pcanet import PCANet
from pcanet import PCANet
import numpy as np
%matplotlib inline

In [2]:
# set cwd back to default
os.chdir('../')
os.getcwd()

'/Users/mark/src/dog-breed-identification/src'

In [3]:
# custom scripts
import config # params, constants
import data, models # functions that mutate outr data
# from utils import utils, plot # custom functions, in local environment

 Dataset :: namedtuple(
    ['train' = ['img_name']
    , 'test' = ['img_name']
    , 'validation' = ['img_name']
    , 'labels' = pandas.df('img_name','label')
    , 'dict_index_to_label' = dict to convert label_index -> label_name
    , 'dict_label_to_index'= dict to convert label_name -> label_index
    


In [4]:
import data # src/data.py
dataset = data.init_dataset()

In [6]:
import os
os.listdir('../models')

['cnn_transfer',
 'cnntransfer.json',
 'cnntransfer2.json',
 'cnntransfer_augm.json',
 'cnntransfer_augm2.json',
 'cnntransferweights.h5',
 'cnntransferweights2.h5',
 'cnntransferweights_augmen.h5',
 'cnntransferweights_augmen2.h5']

### load a model

In [12]:
# load json and create model
def load_model():
    filename = 'cnntransfer.json'
    with open(dirname + filename, 'r') as json: # cnn_transfer_augm
        loaded_model_json = json.read()

In [21]:
# json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("../models/cnntransferweights.h5") # cnntransferweights_augmen.h5
print("Loaded model from disk")

Loaded model from disk


In [68]:
# pick the n classes with the most occuring instances
amt = 5
classes = data.top_classes(dataset.labels, amt)
classes

['paper', 'glass', 'plastic', 'metal', 'cardboard']

In [69]:
maxx = 100
max_train = 100
x_test, n = data.extract_topx_classes(dataset, classes, 'test', maxx, max_train)
n

100

In [70]:
x_test, y_test, n = data.extract_all_test(dataset, x_test)

extract all data: 500


In [71]:
# y_train, y_test, y_validation = data.labels_to_vectors(dataset, y_train, y_test, y_validation)
y_test = data.one_hot(y_test)
input_shape = y_test.shape[1:] # = shape of an individual image (matrix)
output_length = (y_test[0]).shape[0] # = length of an individual label
output_length

5

In [72]:
model_final_augmentation = loaded_model

In [73]:
optimizer = optimizers.Adam(lr=0.001)

In [74]:
model_final_augmentation.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy',
        'mean_squared_error','categorical_crossentropy','top_k_categorical_accuracy'])

## running tests


In [75]:
# import sklearn.metrics.confusion_matrix

def evaluate(model):
    cvscores = []
    scores = model.evaluate(x_test, y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

# evaluate(model_final_augmentation)

acc: 82.40%
82.40% (+/- 0.00%)


In [76]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_test, y_pred)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names,
                          title='Confusion matrix, without normalization')

    # Plot normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                          title='Normalized confusion matrix')

    plt.show()

In [78]:
model_augment = config.dataset_dir + 'models/cnntransfer_augm.json'
model_augment_weights = config.dataset_dir + 'models/cnntransferweights_augmen.h5'
model_default = config.dataset_dir + 'models/cnntransfer.json'
model_default_weights = config.dataset_dir + 'models/cnntransferweights.h5'

# augment = load_model(model_augment, model_augment_weights)
default = load_model(model_default, model_default_weights)
augment = load_model(model_augment, model_augment_weights)



NameError: name 'load_model' is not defined

In [77]:
import tensorflow as tf

y_pred_class = model_final.predict(x_test)
# con = tf.confusion_matrix(labels=y_test, predictions=y_pred_class )
# print(con)

y_test_non_category = [ np.argmax(t) for t in y_test ]
y_predict_non_category = [ np.argmax(t) for t in y_pred_class ]


from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test_non_category, y_predict_non_category)
print(conf_mat)
# def confusion_matrix(model, x_test)
#     y_pred_class = model_final.predict(X_test)
#     con = tf.confusion_matrix(labels=y_train, predictions=y_pred_class )
#     print(cont)

NameError: name 'model_final' is not defined

In [ ]:
num_classes = 5

# from lable to categorial
y_categorical = y_pred_class
# from categorial to lable indexing
y_pred = y_categorial.argmax(1)
print(y_pred)
# plot_confusion_matrix(conf_mat, classes)

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=conf_mat)

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
fig, ax = plot_confusion_matrix(conf_mat=conf_mat)